In [ ]:
pip install biopython


In [ ]:
from Bio import Entrez
import xml.etree.ElementTree as ET
import csv
import time
import pandas as pd
from tqdm import tqdm

In [ ]:
api_key ="xxxxx"
Entrez.email = "xxx"

# Provide the API key to Entrez
Entrez.api_key = api_key

In [ ]:
excel_file="All genes.xlsx"
xls = pd.ExcelFile(excel_file)
sheets_dict = pd.read_excel(xls, sheet_name=None)
# Initialize a list to hold all the gene IDs from all sheets
gene_ids = []

# Extract GeneIDs from each sheet and append to the list
for sheet_name, sheet_data in sheets_dict.items():
    if 'GeneID' in sheet_data.columns:
        gene_ids.extend(sheet_data['GeneID'].tolist())

# Remove duplicate gene IDs, if any
gene_ids = list(set(gene_ids))

In [ ]:
len(gene_ids)

6380

In [ ]:
import time
from tqdm import tqdm
import xml.etree.ElementTree as ET
from Bio import Entrez

# Function to fetch gene information with exponential backoff and retries
def fetch_gene_info_with_retry(gene_ids, max_retries=5, base_delay=1):
    retry_delay = base_delay
    for attempt in range(max_retries):
        try:
            handle = Entrez.efetch(db="gene", id=gene_id, rettype="xml")
            xml_data = handle.read()
            handle.close()
            return xml_data, None  # Return xml_data and no error
        except Exception as e:
            print(f"Error occurred for gene ID {gene_id}: {e}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
    return None, f"Failed to fetch gene information for gene ID {gene_id} after {max_retries} retries."

# Initialize the list to store gene data
gene_data = []


# Initialize tqdm progress bar
progress_bar = tqdm(total=len(gene_ids), desc="Fetching Gene Information")

# Fetch information for each geneID from NCBI
for gene_id in gene_ids:
    xml_data, error = fetch_gene_info_with_retry(gene_id)  # Fetch gene info with retries

    if error:
        print("Error occurred for gene ID {}: {}".format(gene_id, error))
        continue  # Skip to the next gene ID if an error occurs

    # Parse XML data
    root = ET.fromstring(xml_data)

    # Navigate the XML to extract needed information
    for gene in root.findall('.//Entrezgene'):
        gene_id = gene.find('.//Gene-track_geneid').text
        status = gene.find('.//Gene-track_status').attrib.get('value', '')  # Use get method to handle None
        gene_name = gene.find('.//Gene-ref_locus').text if gene.find('.//Gene-ref_locus') is not None else ""
        description = gene.find('.//Gene-ref_desc').text if gene.find('.//Gene-ref_desc') is not None else ""
        summary = gene.find('.//Entrezgene_summary').text if gene.find('.//Entrezgene_summary') is not None else ""

        # Check if any of the attributes is None, and if so, skip this gene
        if None in [gene_id, status, gene_name, description, summary]:
            print(f"One or more attributes missing for gene ID {gene_id}. Skipping.")
            continue

        synonyms = [syn.text for syn in gene.findall('.//Gene-ref_syn')]
        synonym_text = ', '.join(synonyms)

        gene_data.append({
            "Gene ID": gene_id,
            "Status": status,
            "Gene Name": gene_name,
            "Description": description,
            "Summary": summary,
            "Synonyms": synonym_text
        })

    # Update progress bar
    progress_bar.update(1)

    time.sleep(0.1)  # Sleep for 0.1 second (10 requests per second)

# Close tqdm progress bar
progress_bar.close()

# Convert the list of dictionaries into a DataFrame
import pandas as pd
df_gene_info = pd.DataFrame(gene_data)
print(df_gene_info)


Fetching Gene Information:   1%|▏         | 80/6380 [00:40<53:50,  1.95it/s]

Error occurred for gene ID CDZ96283: HTTP Error 400: Bad Request. Retrying in 1 seconds...


Fetching Gene Information:   2%|▏         | 156/6380 [01:15<29:22,  3.53it/s]

Error occurred for gene ID CDZ97970: HTTP Error 400: Bad Request. Retrying in 1 seconds...


Fetching Gene Information:   3%|▎         | 206/6380 [01:45<40:50,  2.52it/s]

...

Error occurred for gene ID CDZ96423: HTTP Error 400: Bad Request. Retrying in 1 seconds...


Fetching Gene Information:  99%|█████████▉| 6323/6380 [49:05<00:18,  3.09it/s]

Error occurred for gene ID CDZ98076: HTTP Error 400: Bad Request. Retrying in 1 seconds...


Fetching Gene Information: 100%|██████████| 6380/6380 [49:30<00:00,  2.15it/s]

     Gene ID        Status      Gene Name  \
0      97577     secondary  E230019G03Rik   
1      97252     secondary         C81327   
2      96289  discontinued       LOC96289   
3      84545          live         MRPL43   
4      82891  discontinued       LOC82891   
...      ...           ...            ...   
6310   84278          live       MFSD14CP   
6311   96658  discontinued       LOC96658   
6312   96793  discontinued       LOC96793   
6313   97854     secondary         C77215   
6314   98269     secondary       AI132431   

                                            Description  \
0                            RIKEN cDNA E230019G03 gene   
1                             expressed sequence C81327   
2     similar to enabled homolog (Drosophila) (M. mu...   
3                   mitochondrial ribosomal protein L43   
4     similar to RED PROTEIN (RER PROTEIN) (M. muscu...   
...                                                 ...   
6310  major facilitator superfamily domain con

In [ ]:
df_gene_info.to_excel('df_gene_info.xlsx')

In [ ]:
df_gene_info

,Gene ID,Status,Gene Name,Description,Summary,Synonyms
0,97577,secondary,E230019G03Rik,RIKEN cDNA E230019G03 gene,,
1,97252,secondary,C81327,expressed sequence C81327,,
2,96289,discontinued,LOC96289,similar to enabled homolog (Drosophila) (M. mu...,DISCONTINUED: This record has been withdrawn b...,
3,84545,live,MRPL43,mitochondrial ribosomal protein L43,Mammalian mitochondrial ribosomal proteins are...,\n
4,82891,discontinued,LOC82891,similar to RED PROTEIN (RER PROTEIN) (M. muscu...,DISCONTINUED: This record has been withdrawn b...,
...,...,...,...,...,...,...
6310,84278,live,MFSD14CP,major facilitator superfamily domain containin...,Predicted to enable transmembrane transporter ...,\n
6311,96658,discontinued,LOC96658,similar to ZINC FINGER PROTEIN 23 (ZINC FINGER...,DISCONTINUED: This record has been withdrawn b...,
6312,96793,discontinued,LOC96793,similar to hypothetical protein MGC2615 (H. sa...,DISCONTINUED: This record has been withdrawn b...,
6313,97854,secondary,C77215,expressed sequence C77215,,


In [ ]:
excel_file_path = 'All genes.xlsx'
excel_sheets = pd.ExcelFile(excel_file_path)

In [ ]:
# Initialize an ExcelWriter to write back to the Excel file
with pd.ExcelWriter('updated.xlsx', engine='openpyxl') as writer:
    for sheet_name in excel_sheets.sheet_names:
        # Read the current sheet into a DataFrame
        df_sheet = pd.read_excel(excel_sheets, sheet_name=sheet_name)

        # Extract numerical ID from 'GeneID' in the sheet for matching
        df_sheet['Numerical_ID'] = df_sheet['GeneID'].astype(str).str.extract('(\d+)').astype(int)

        # Merge the current sheet with the 'df' DataFrame based on numerical IDs
        df_merged = df_sheet.merge(df, left_on='Numerical_ID', right_on='GeneID', how='left')

        # Safely drop the 'Numerical_ID' column if it exists
        if 'Numerical_ID' in df_merged.columns:
            df_merged.drop('Numerical_ID', axis=1, inplace=True)

        # Write the merged DataFrame to the ExcelWriter object
        df_merged.to_excel(writer, index=False, sheet_name=sheet_name)
